<a href="https://colab.research.google.com/github/krahul2024/machine-learning/blob/main/projects/next_word_predict/next_word_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Imports
import tensorflow as tf,  pandas as pd, numpy as np, nltk
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
tokenizer = Tokenizer()

In [4]:
nltk.download('brown')
from nltk.corpus import brown

text_data = ' '.join(brown.words())
len(text_data)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


6127073

In [5]:
# assign each word a number, as this helps in generating the outputs

tokenizer.fit_on_texts([text_data])
indexes = tokenizer.word_index
len(indexes)

44541

In [6]:
nltk.download('punkt')
sents = nltk.sent_tokenize(text_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
'''
From all these single word tokens and sentence tokens what we have to create now is ,
let's say a sentence  .... this is cat and it is blue .... this : 23, is : 32, cat : 433  and so on....--> word : index
then we have to prepare the dataset as shown below,

this -> is
this is -> cat
this is cat -> and
this is cat and -> it
....

this is how our prediction works when we type in the keyboard,
just that now in place of these words we have to generate the dataset in terms of indexes of these words in our list of word
indexes , as computer don't understand the english languages, heck they don't even understand string just numbers.
'''
# converting the sentences which are made up of words into group of indexes of the words they are made up of
x, y = [], []
for sent in sents :
    sent_tokens = tokenizer.texts_to_sequences([sent])[0]
    for i in range(1, len(sent_tokens)):
        input_seq = sent_tokens[:i+1]
        x.append(input_seq[:-1])
        y.append(input_seq[-1])

x = np.array(x)
y = np.array(y)


<ipython-input-7-7d6ced0ce831>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [8]:
for i in range(10):
    print(x[i], '--->', y[i])

[1] ---> 5514
[1, 5514] ---> 658
[1, 5514, 658] ---> 2189
[1, 5514, 658, 2189] ---> 1652
[1, 5514, 658, 2189, 1652] ---> 55
[1, 5514, 658, 2189, 1652, 55] ---> 1901
[1, 5514, 658, 2189, 1652, 55, 1901] ---> 30
[1, 5514, 658, 2189, 1652, 55, 1901, 30] ---> 2220
[1, 5514, 658, 2189, 1652, 55, 1901, 30, 2220] ---> 2
[1, 5514, 658, 2189, 1652, 55, 1901, 30, 2220, 2] ---> 14185


In [9]:
# We have to find the longest sentence and then make each array of that length by padding some value: 0, because when we
# do the training then we need constant size of the matrix or the input size should be consistent
max_len = max([len(arr) for arr in x])
max_len

333

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_seq = pad_sequences(x, maxlen = max_len, padding = 'pre') # padding at the begining because we don't have anything indexed with 0 and also we don't want to output nothing.
pad_seq

array([[    0,     0,     0, ...,     0,     0,     1],
       [    0,     0,     0, ...,     0,     1,  5514],
       [    0,     0,     0, ...,     1,  5514,   658],
       ...,
       [    0,     0,     0, ..., 11491,  2596, 44540],
       [    0,     0,     0, ...,  2596, 44540,  1681],
       [    0,     0,     0, ..., 44540,  1681,     9]], dtype=int32)

In [11]:
len(pad_seq)

979759

In [12]:
# we can treat this problem as classification problem not a regression task, thus we have to transform the output using one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes = len(indexes)+1)
y.shape

(979759, 44542)